In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import re
import requests

In [16]:
def getdriver(): #it is the selenium driver to connect the script with the browser
    options = webdriver.ChromeOptions()
    options.add_argument('log-level=3')
    options.add_argument("--window-size=1880x1020")
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    #options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("test-type")
    options.add_argument("--disable-web-security")
    options.add_argument("--allow-running-insecure-content")
    #options.add_argument('--proxy-server=%s' % proxy)
    driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())
    #     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """})
    return driver
driver = getdriver()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Pfactorial00110\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_4192\459452054.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())


In [87]:
job_link = []
Category = []
#the argument url takes website url as parameter eg:'https://www.careerjet.co.in/machine-learning-jobs.html?p='
#the argument category takes category of jobs as parameter
def hirist_extractor(url, category):#this functions extracts the job links from the particular websie
  try:
    url_next=url
    driver.get(url_next) # to access the url
    content = driver.page_source
    page = BeautifulSoup(content, "html.parser")
    pages = page.findAll(class_="job-title")
    for ele in pages:
      a = ele.find('a')
      if a==None:
        pass
      elif a['href'].startswith('/j/'):
        job_link.append("https://www.hirist.com"+a['href']) # adds the iterted data to the empty list
        Category.append(category)
  except:
    pass
  return job_link,Category

In [80]:
job_link,Category = hirist_extractor("https://www.hirist.com/search/Data%20Science-0-0-900-10-5-871-0-2-2.html", "Data Science")

In [88]:
job_link,Category = hirist_extractor("https://www.hirist.com/search/Machine%20Learning-0-0-300-4-5-139-0-2-2.html", "Machine Learning")

In [90]:
Job_posting_df=pd.DataFrame({"Category":Category,"Job_Posting_link":job_link})

In [83]:
Job_posting_df.to_csv('Data_science_hirist_links.csv', index=False)

In [91]:
Job_posting_df.to_csv('Machine_learning_hirist_links.csv', index=False)

In [3]:
ds = pd.read_csv("Data_science_hirist_links.csv")
ml = pd.read_csv('Machine_learning_hirist_links.csv')

In [32]:
def hirist_details_extractor(df): # this function extracts the data from the particular job link
    hirist_df = pd.DataFrame()
    for i in range(0,1): #it iterates through the links upto the extracted links length
        try:
          Job_Posting_link=df['Job_Posting_link'][0] #iterates through the links in the dataframe
          Category=df['Category'][0]
          driver.get(Job_Posting_link)
          try:
            Location=driver.find_element(by=By.XPATH, value='//*[@id="company-location-tooltip"]/a').text #extracts the text data from the respective class                                   #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[1]/p[2]
          except:
            Location=None
          try:                                                
            company = driver.find_element(by=By.XPATH, value='//*[@id="root"]/div/div/div/div[2]/div[2]/div/div[2]/a[1]').text #soups1.find(class_ = 'info-org').text
          except:
            company=None
          try:
            skills = driver.find_element(by=By.XPATH, value='//*[@id="root"]/div/div/div/div[2]/div[2]/div/div[3]').text #('//div[@class="col"][3]').text #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]
          except:
            skills = None
          try:
            title = driver.find_element(by=By.CLASS_NAME, value="job-detail-title").text #soups1.find(class_ = 'info-position').text
          except:
            title=None
          try:
            salary = driver.find_element(by=By.XPATH, value='//*[@id="jobDescription"]/div[1]/section[1]/div[2]/div[1]/div[1]/div[1]/p[2]').text
          except:                                             
            salary=None
          try:                                                    
            description = driver.find_element(by=By.XPATH, value='//*[@id="root"]/div/div/div/div[3]/div[1]/div[1]/div[1]/p').text #soups1.find(class_='job-description').text 
          except:                                                 
            description=None
          try:                                                    
            experience = driver.find_element(by=By.XPATH, value='//*[@id="root"]/div/div/div/div[2]/div[2]/div/div[2]/a[2]').text
          except:
            experience = None
          try:
            education = driver.find_element(by=By.XPATH, value='//*[@id="jobDescription"]/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]').text #('//div[@class="col"][3]').text #/html/body/div[1]/div[2]/main/div[1]/section[1]/div[3]/div[1]/div[1]/div[3]/p[2]
          except:
            education = None
          hirist=pd.DataFrame({'Category':Category,'Job_Posting_link':Job_Posting_link,'Job_title':title,'Company_name':company,'Skills':skills,'Salary':salary,'Location':Location,'Education':education,'Experience':experience,'Description':[description]},index=[0])
          hirist_df=hirist_df.append(hirist)
        except Exception as e: # raises an exception if something goes wrong in the time of extraction
          print(e)
    return hirist_df

In [27]:
data_science = hirist_details_extractor(ds) #calls the above funtion

C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_4192\4226106848.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hirist_df=hirist_df.append(hirist)


In [33]:
machine_learning = hirist_details_extractor(ml) #calls the above funtion

C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_4192\301408012.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hirist_df=hirist_df.append(hirist)


In [34]:
machine_learning

,Category,Job_Posting_link,Job_title,Company_name,Skills,Salary,Location,Education,Experience,Description
0,Machine Learning,https://www.hirist.com/j/computer-vision-inter...,Computer Vision Intern - Deep/Machine Learning...,Mirrorsize US Inc.,Computer VisionInternshipData ScienceMachine L...,None,Noida,None,0 - 1 Years,RESPONSIBILITY :\n\n- Performing tests and imp...


In [10]:
data_science.to_csv("hirist_data_science_full_details.csv", index=False)

In [15]:
machine_learning.to_csv("hirist_Machine_Learning_full_details.csv", index=False)